# Workshop_02

Per lo svolgimento del secono workshop si è deciso di prendere spunto dalla figura sottostante.

<img src="jupyter_images/workshop_02_picture.jpg">

## Dimensioni della figura e file csv
I pilastri e le travi sono supposti essere tutti uguali tra loro e aventi ognuno il lato più corto di dimensioni 1.5x1.5.
Con riferimento al primo telaio le distanze tra i vari pilastri e tra le varie travi sono state supposte essere 10, mentre le distanze tra i vari telai sono state supposte essere 15.

Tutte le distanze di cui sopra sono elencate nel file frame_data_457856.csv (riportato in seguito), con la seguente convenzione:
* le righe di posto pari indicano le dimsioni dei telai;  
* le righe di posto dispari indicano la distanza tra i telai riportati nella riga superiore e inferiore.

Nelle righe di posto pari i dati sono articolati come segue:
"bx,bz,[array con le distanze degli assi dei pilastri],px,py,[array con le distanze interpiano]",
con bx,bz dimensioni della sezione di una trave e px,py dimensioni della sezione di un pilastro.

1.5,1.5,[,10,10,],1.5,1.5,[,10,10,10,10,10,10,]  
15  
1.5,1.5,[,10,10,],1.5,1.5,[,10,10,10,10,10,10,]  
15  
1.5,1.5,[,10,10,],1.5,1.5,[,10,10,10,10,10,10,]  
15  
1.5,1.5,[,10,10,],1.5,1.5,[,10,10,10,10,10,10,]  
15  
1.5,1.5,[,10,10,],1.5,1.5,[,10,10,10,10,10,10,]

## Lettura del file csv
Per la lettura del file csv di cui sopra è stata definita la seguente funzione:

In [10]:
"""read data from a csv file"""
def readCSV(file_name):
	
	frameDist = []
	with open(file_name, 'rb') as csvfile:
		reader = csv.reader(csvfile)
		i=0
		for row in reader:
			if i%2==1:
				frameDist.append(float(row[0]))
				i=i+1
			elif i%2==0:
				(px,py)=(float(row[0]),float(row[1]))
				pillarsDist = []
				j=3
				while row[j]!="]":
					pillarsDist.append(float(row[j]))
					j=j+1
				(by,bz)= (float(row[j+1]),float(row[j+2]))
				j= j+4
				beamsDist = []
				while row[j]!="]":
					beamsDist.append(float(row[j]))
					j=j+1	
				i=i+1
		csvfile.close()
        
	return (px,py, pillarsDist, by,bz, beamsDist, frameDist)

## Creazione della struttura
Per la creazione dela struttura è stata definita la funzione riportata in seguito.

Il codice si articola in due sezioni principali, identificate da opportuni commenti. Nella prima sezione viene creato un singolo telaio (frame), mentre nella seconda vengono replicati i telai e create le travi tra le varie copie dei telai stessi. Infine viene combinato il tutto per ottenere la strutura finale.

In [3]:
"""return a 3D value of type HPC of the structure with the given parameters"""
def buildStruct(px,py,pillarsDist,by,bz, beamsDist, frameDist):

	#the following code creates a single frame
	pillarArray = [px]
	for value in pillarsDist:
		pillarArray = pillarArray+[-value]+[px]
	xPillar = QUOTE(pillarArray)
	yPillar = QUOTE([py])
	xyPillar = PROD([xPillar, yPillar])
	heightsPillar = []
	for value in beamsDist:
		heightsPillar.append((value+bz))
	zPillar = QUOTE(heightsPillar)
	pillars = PROD([xyPillar, zPillar])
	beamArray = []
	for value in pillarArray:
		beamArray.append(-value)
	xBeam = QUOTE(beamArray)
	yBeam = QUOTE([by])
	xyBeam = PROD([xBeam, yBeam])
	beamsArrayDist = []
	for value in beamsDist:
		beamsArrayDist.append(-value)
		beamsArrayDist.append(bz)
	beams = PROD([xyBeam, QUOTE(beamsArrayDist)])
	#the single frame created
	frame = STRUCT([pillars, beams])
	
	#the following code duplicates the frames and creates the beams between each frame
	framesArrayDist = []
	for value in frameDist:
		framesArrayDist.append(-py)
		framesArrayDist.append(value)
	yFrameBeams = QUOTE(framesArrayDist)
	xyFrameBeams = PROD([yBeam, yFrameBeams])
	xyzFrameBeams = PROD([xyFrameBeams, QUOTE(beamsArrayDist)])
	xyzFrame = STRUCT([xyzFrameBeams])
	
	accB = 0
	arrayBeams = []
	for value in frameDist:
		accB+=value
		arrayBeams.append(accB)
	distB=py
	structFrames=frame
	for value in arrayBeams:
		structFrames = STRUCT([structFrames, T(2)(value+distB), frame])
		distB+=py

	accP = 0
	arrayPillar = []
	for value in pillarsDist:
		accP+=value
		arrayPillar.append(accP)
	distP=px
	structBeams= xyzFrame
	for value in arrayPillar:
		structBeams = STRUCT([structBeams, T(1)(value+distP), xyzFrame])
		distP+=px

	#merge frame and beams to create the final structure
	finalStruct =  STRUCT([structFrames, structBeams])

	return finalStruct

## Programma finale
Le due funzioni precedenti sono state combinate tramite la seguente funzione "ggpl_bone_structure(csvFileName)" per ottenere i risultato desiderato.
Prima vengono recuperati i dati dal file csv e memorizzati in un adeguata struttura dati "structureDati", così da poter in seguito estrarre i singoli parametri da passare alla funzione "buildStruct", la quale crea la struttura in 3D, visualizzata infine tramite l'ultima istruzione.

In [4]:
"""visualize a type HPC of a 3D structure. The data to build the structure are in a csv file given in input"""
def ggpl_bone_structure(csvFileName):

	#read data from the csv file and save them in structureData
	structureData = readCSV(csvFileName)
	
	#extract eache single parameter for buildStruct from structureData
	px = structureData[0]
	py = structureData[1]
	pillarDist = structureData[2]
	by = structureData[3]
	bz = structureData[4]
	beamDist = structureData[5]
	frameDist = structureData[6]

	#create the structure
	struct = buildStruct(px,py,pillarDist,by,bz,beamDist,frameDist)
	
	#visualize the structure
	VIEW(struct)

### Di seguito sono riportate immagini dei passaggi significativi svolti durante la creazione della struttura finale:

#### Singolo telaio
<img src="jupyter_images/workshop_02_telaio.png">

#### Vari telai replicati
<img src="jupyter_images/workshop_02_telai.png">

#### Travi tra telai
<img src="jupyter_images/workshop_02_travi.png">

## Il risultato finale ottenuto  è il seguente:

<img src="jupyter_images/workshop_02_struttura.png">

### Viene infine riportato il codice completo dell'applicazione creata:

In [9]:
from larlib import * 

import csv

"""read data from a csv file"""
def readCSV(file_name):
	
	frameDist = []
	with open(file_name, 'rb') as csvfile:
		reader = csv.reader(csvfile)
		i=0
		for row in reader:
			if i%2==1:
				frameDist.append(float(row[0]))
				i=i+1
			elif i%2==0:
				(px,py)=(float(row[0]),float(row[1]))
				pillarsDist = []
				j=3
				while row[j]!="]":
					pillarsDist.append(float(row[j]))
					j=j+1
				(by,bz)= (float(row[j+1]),float(row[j+2]))
				j= j+4
				beamsDist = []
				while row[j]!="]":
					beamsDist.append(float(row[j]))
					j=j+1	
				i=i+1
		csvfile.close()
		
	return (px,py, pillarsDist, by,bz, beamsDist, frameDist)

"""return a 3D value of type HPC of the structure with the given parameters"""
def buildStruct(px,py,pillarsDist,by,bz, beamsDist, frameDist):

	#the following code creates a single frame
	pillarArray = [px]
	for value in pillarsDist:
		pillarArray = pillarArray+[-value]+[px]
	xPillar = QUOTE(pillarArray)
	yPillar = QUOTE([py])
	xyPillar = PROD([xPillar, yPillar])
	heightsPillar = []
	for value in beamsDist:
		heightsPillar.append((value+bz))
	zPillar = QUOTE(heightsPillar)
	pillars = PROD([xyPillar, zPillar])
	beamArray = []
	for value in pillarArray:
		beamArray.append(-value)
	xBeam = QUOTE(beamArray)
	yBeam = QUOTE([by])
	xyBeam = PROD([xBeam, yBeam])
	beamsArrayDist = []
	for value in beamsDist:
		beamsArrayDist.append(-value)
		beamsArrayDist.append(bz)
	beams = PROD([xyBeam, QUOTE(beamsArrayDist)])
	#the single frame created
	frame = STRUCT([pillars, beams])
	
	#the following code duplicates the frames and creates the beams between each frame
	framesArrayDist = []
	for value in frameDist:
		framesArrayDist.append(-py)
		framesArrayDist.append(value)
	yFrameBeams = QUOTE(framesArrayDist)
	xyFrameBeams = PROD([yBeam, yFrameBeams])
	xyzFrameBeams = PROD([xyFrameBeams, QUOTE(beamsArrayDist)])
	xyzFrame = STRUCT([xyzFrameBeams])
	
	accB = 0
	arrayBeams = []
	for value in frameDist:
		accB+=value
		arrayBeams.append(accB)
	distB=py
	structFrames=frame
	for value in arrayBeams:
		structFrames = STRUCT([structFrames, T(2)(value+distB), frame])
		distB+=py

	accP = 0
	arrayPillar = []
	for value in pillarsDist:
		accP+=value
		arrayPillar.append(accP)
	distP=px
	structBeams= xyzFrame
	for value in arrayPillar:
		structBeams = STRUCT([structBeams, T(1)(value+distP), xyzFrame])
		distP+=px

	#merge frame and beams to create the final structure
	finalStruct =  STRUCT([structFrames, structBeams])

	return finalStruct

"""visualize a type HPC of a 3D structure. The data to build the structure are in a csv file given in input"""
def ggpl_bone_structure(csvFileName):

	#read data from the csv file and save them in structureData
	structureData = readCSV(csvFileName)
	
	#extract eache single parameter for buildStruct from structureData
	px = structureData[0]
	py = structureData[1]
	pillarDist = structureData[2]
	by = structureData[3]
	bz = structureData[4]
	beamDist = structureData[5]
	frameDist = structureData[6]

	#create the structure
	struct = buildStruct(px,py,pillarDist,by,bz,beamDist,frameDist)
	
	#visualize the structure
	VIEW(struct)

def main():
	ggpl_bone_structure('frame_data_457856.csv')

if __name__=='__main__':
	main()